S1 "Suit of card #1"
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

2) C1 "Rank of card #1"
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

3) S2 "Suit of card #2"
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

4) C2 "Rank of card #2"
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

5) S3 "Suit of card #3"
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

6) C3 "Rank of card #3"
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

7) S4 "Suit of card #4"
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

8) C4 "Rank of card #4"
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

9) S5 "Suit of card #5"
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

10) C5 "Rank of card 5"
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

11) CLASS "Poker Hand"
Ordinal (0-9)

0: Nothing in hand; not a recognized poker hand
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush



Ulazni podaci će nam biti u obliku niza od 85 elemenata,koji se sastoji samo od nula i jedinica.
Zašto 85 elemenata? Svaka ruka u pokeru se sastoji od 5 karata. Svaka karta ima broj i 'boju'. Što bi značilo da se za označavanje svake karte koristi 17 brojeva unutar niza 17*5=85.

Prvih 17 brojeva označava prvu kartu,drugih 17 drugu itd.
Zašto 17 brojeva? Zato što se svaka karta sastoji od 'boje'(ukupno imamo 4 boje) i broja(13 različitih brojeva).
Prva 4 broja, u nizu od 17 elemenata, služe za označavanje 'boje' a sljedećih 13 za označavanje broja karte.

Ukoliko imamo AS jedinica će biti na prvom mjestu a na ostalih 12 mjesta će biti nula. Za broj 5 na 5-tom mjestu ćemo imati jedinicu dok će na ostalima biti nula.
Kod 'boja' ako imamo Hearts jedinica je na prvom mjesta a ostala 3 polja su popunjena s nulom. Za Spades jedinica je na drugom mjestu, a ostala polja imaju vrijednost nula. Diamonds ima jedinicu na 3 trećem mjestu, a Clubs na zadnjem.

Npr. 2Hearts ->1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0(brojevi na pozicijama od 0-3 služe za označavanje boje,a od 4-16 za broj karte)

     11Diamonds->0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
     
Na isti način pretvaramo i izlaz. Pošto imamo 10 mogućih izlaza niz se sastoji od 10 elemenata. 
Ništa u ruci će na prvom mjestu imati jedinicu a ostalih 9 mjesta će biti popunjeno s nulom itd..

In [2]:
import pandas as pd
import numpy as np
import unicodecsv
import csv

In [3]:
df = pd.read_csv('poker_training_data.csv',delimiter=',')
dt=pd.read_csv('poker_test_data.csv',delimiter=',')

def convert_to_binaryarray(data):
    cards=[]
    hand=[]
    for el in data.values:
        cards.append(el[:-1].tolist())
        hand.append(el[-1])
    
    #pretvaranje karata u niz binarnih brojeva
    suit=[0,0,0,0]
    rank=[0,0,0,0,0,0,0,0,0,0,0,0,0]

    data_set=[]
    for el in cards:
        h=[]
        for i in range(len(el)):
            if(i%2==0):
                suit[el[i]-1]=1
                h+=suit
                suit=[0,0,0,0]
            else:
                rank[el[i]-1]=1 
                h+=rank
                rank=[0,0,0,0,0,0,0,0,0,0,0,0,0]
        data_set.append(h)
        
    #pretvaranje iz liste u niz
    a=[]
    for el in data_set:
        a.append(np.array(el))
    
    a=np.array(a)
    
    #pretvaranje rjesenja u niz binarnih brojeva
    o=[0,0,0,0,0,0,0,0,0,0]

    out=[]
    for el in hand:
        o[el]=1
        out.append(o)
        o=[0,0,0,0,0,0,0,0,0,0]
    
    return a,out
    

In [4]:
def convert_to_result(rj):
    rjesenje=[]
    for i in range(len(rj)):
        for j in range(10):
            if(rj[i][j]>0.5):
                rj[i][j]=1
            else:
                rj[i][j]=0
        
        nule=np.count_nonzero(rj[i]==0)
        jedinice=np.count_nonzero(rj[i]==1)
    
        if(nule+jedinice>10):
            print('pogreska')
        if(jedinice==1):
            a=np.where(rj[i]==1)
            rjesenje.append(a[0])
        else:
            rjesenje.append(rj[i])
    return rjesenje
            
    

In [ ]:
#PREDVIDANJE
#Multilayer perceptron with input(85),output(10) and 2 hidden layers(18,10)

In [5]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_d(x):
    return (x)*(1.0-(x))

In [21]:
#treniranje

lr=0.5

w0=np.random.random((85,18))
w1=np.random.random((18,10))
w2=np.random.random((10,10))

training_data,training_data_result=convert_to_binaryarray(df)

for i in range(200):
    for i in range(len(training_data)):
        layer0=np.array([training_data[i]])#niz iz data_set-a smo morali staviti unutar niza(da bi dobili matricu),tako da bi mogli transponirati(radi mnozenja)
        layer1=sigmoid(np.dot(layer0,w0))
        layer2=sigmoid(np.dot(layer1,w1))
        output=sigmoid(np.dot(layer2,w2))

        output_er=output-training_data_result[i]
        output_del=output_er*sigmoid_d(output)

        layer2_del=(output_del.dot(w2.T))*sigmoid_d(layer2)

        layer1_del=(layer2_del.dot(w1.T))*sigmoid_d(layer1)

        layer0_del=(layer1_del.dot(w0.T))*sigmoid_d(layer0)

        w0=w0-(lr*layer0.T.dot(layer1_del))
        w1=w1-(lr*layer1.T.dot(layer2_del))
        w2=w2-(lr*layer2.T.dot(output_del))


In [38]:
#testiranje

test_data,test_data_result=convert_to_binaryarray(dt)

layer_0 = test_data
layer_1 = sigmoid(np.dot(layer_0,w0))
layer_2 = sigmoid(np.dot(layer_1,w1))
layer_3 = sigmoid(np.dot(layer_2,w2))

layer_3=convert_to_result(layer_3)

isti=0
for i in range(len(layer_3)):
    if(layer_3[i][0]==dt.values[i][-1]):
        isti+=1
        
postotak=(isti/len(layer_3))*100
print('od ',len(layer_3),' istanci njih ',isti,' je isto. NN radi s točnošću od:',round(postotak,2),'%')

od  26714  istanci njih  24823  je isto. NN radi s točnošću od: 92.92 %
